In [111]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data_mil = pd.read_csv('./data/STRIKE_REPORTS_BASH (1990-Current).csv')
data_90 = pd.read_csv('./data/STRIKE_REPORTS (1990-1999).csv', encoding = "ISO-8859-1")
data_00 = pd.read_csv('./data/STRIKE_REPORTS (2000-2009).csv', encoding = "ISO-8859-1")
data_10 = pd.read_csv('./data/STRIKE_REPORTS (2010-Current).csv', encoding = "ISO-8859-1")
data = pd.concat([data_90, data_00, data_10, data_mil])
data.head()

/home/shlapo/.anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,7,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/shlapo/.anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/shlapo/.anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,7,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,INDEX_NR,OPID,OPERATOR,ATYPE,AMA,AMO,EMA,EMO,AC_CLASS,AC_MASS,...,REPORTED_NAME,REPORTED_TITLE,REPORTED_DATE,SOURCE,PERSON,NR_INJURIES,NR_FATALITIES,LUPDATE,TRANSFER,INDICATED_DAMAGE
0,100000,AAL,AMERICAN AIRLINES,B-727,148,10,34.0,10,A,4.0,...,Deleted,Deleted,5/14/1992,FAA Form 5200-7,Pilot,NaN,NaN,3/14/1996,False,False
1,100001,UAL,UNITED AIRLINES,B-737-300,148,24,10.0,01,A,4.0,...,Deleted,Deleted,NaN,Air Transport Report,Air Transport Operations,NaN,NaN,12/20/2007,False,False
2,100002,UAL,UNITED AIRLINES,B-737-300,148,24,10.0,01,A,4.0,...,Deleted,Deleted,NaN,Air Transport Report,Air Transport Operations,NaN,NaN,12/20/2007,False,False
3,100003,UAL,UNITED AIRLINES,B-757-200,148,26,34.0,40,A,4.0,...,Deleted,Deleted,NaN,Air Transport Report,Air Transport Operations,NaN,NaN,12/20/2007,False,False
4,100004,UAL,UNITED AIRLINES,A-320,04A,03,23.0,01,A,4.0,...,Deleted,Deleted,NaN,Air Transport Report,Air Transport Operations,NaN,NaN,12/20/2007,False,False


In [141]:
# Correction for sky
data.SKY.replace(['No CLoud', 'NO Cloud', 'No Clouds'], 'No Cloud', inplace=True)
data.SKY.replace(['SOme Cloud', 'Some Clouds'], 'Some Cloud', inplace=True)
data.SKY.value_counts()

No Cloud      42951
Some Cloud    29580
Overcast      15635
Name: SKY, dtype: int64

### Columns to Investigate

- Month, Year
- State
- Height


In [38]:
data_freq = pd.crosstab(index=data.BIRDS_STRUCK, columns=data.BIRDS_SEEN)
data_freq.drop('Over 100', inplace=True)
data_freq.drop('0', axis=1, inplace=True)
data_freq

BIRDS_SEEN,1,11-100,2-10,Over 100
BIRDS_STRUCK,,,,
1,35639,2170,11777,209
11-100,16,641,35,142
2-10,514,2424,8982,292


In [46]:
import scipy.stats as stats
results = stats.chi2_contingency(observed=data_freq)
results[3]

array([[  2.86601957e+04,   4.14819664e+03,   1.64770966e+04,
          5.09511068e+02],
       [  4.80020146e+02,   6.94767747e+01,   2.75969447e+02,
          8.53363250e+00],
       [  7.02878420e+03,   1.01732659e+03,   4.04093391e+03,
          1.24955300e+02]])

In [45]:
expected = results[3]
(data_freq - expected)/(expected**.5)

BIRDS_SEEN,1,11-100,2-10,Over 100
BIRDS_STRUCK,,,,
1,41.223176,-30.714241,-36.615624,-13.313237
11-100,-21.179081,68.566828,-14.505459,45.688287
2-10,-77.706962,44.102487,77.728411,14.943604


In [54]:
month_struck = pd.crosstab(index=data.BIRDS_STRUCK, columns=data.INCIDENT_MONTH)
month_struck.drop('Over 100', inplace=True)

In [58]:
(chi, p, dof, expected) = stats.chi2_contingency(observed=month_struck)
p

7.2513102308696368e-249

In [60]:
(month_struck - expected)/(expected**.5)

INCIDENT_MONTH,1,2,3,4,5,6,7,8,9,10,11,12
BIRDS_STRUCK,,,,,,,,,,,,
1,-4.385337,-2.007332,0.949740,3.467345,6.892614,3.313823,-0.027069,-2.927226,0.007329,1.202083,-5.315190,-5.391401
11-100,1.395712,3.019689,-1.702364,-2.086698,-7.317984,-3.229543,-1.275360,3.044118,0.249048,-0.743422,6.012091,6.832602
2-10,11.030647,4.496952,-2.064201,-8.493556,-16.148842,-7.831000,0.365882,6.873042,-0.076737,-2.939967,12.367508,12.374531


In [175]:
month = pd.crosstab(index=data.INCIDENT_MONTH, columns="strikes")
expected_ratio = pd.DataFrame([0.0805825506,
0.0743081384,
0.0855245698,
0.0829882006,
0.0854051413,
0.0860965958,
0.0897943694,
0.0892407091,
0.0811542073,
0.0837319905,
0.07962685,
0.0815466772], index=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
month['ratio'] = expected_ratio
month['expected'] = month.ratio*[month.strikes.sum()/12]*12
month

col_0,strikes,ratio,expected
INCIDENT_MONTH,,,
1,6057,0.080583,13634.164649
2,5561,0.074308,12572.565477
3,8721,0.085525,14470.329587
4,12637,0.082988,14041.188601
5,16891,0.085405,14450.122882
6,14504,0.086097,14567.113526
7,21910,0.089794,15192.758331
8,23828,0.089241,15099.081776
9,21629,0.081154,13730.886104


In [176]:
stats.chisquare(f_obs=month.strikes, f_exp=month.expected)

Power_divergenceResult(statistic=30010.497779218975, pvalue=0.0)

In [177]:
(month.strikes-month.expected)/(month.expected**.5)

INCIDENT_MONTH
1    -64.892194
2    -62.532104
3    -47.794508
4    -11.850141
5     20.305334
6     -0.522921
7     54.497008
8     71.037089
9     67.402206
10    47.535805
11   -20.569013
12   -61.724628
dtype: float64

Looks like months 5, 6-10 have significant impact on bird strikes. Months have different average weather, so lets see if weather and months have any correlation

In [142]:
month_sky = pd.crosstab(index=data.INCIDENT_MONTH, columns=data.SKY)
month_sky


SKY,No Cloud,Overcast,Some Cloud
INCIDENT_MONTH,,,
1,1418,921,861
2,1353,695,894
3,2267,1049,1379
4,3326,1280,2135
5,4132,1643,3041
6,3234,1084,2702
7,4951,1499,4242
8,5674,1872,4872
9,6115,1715,3720


In [146]:
(chi, p, dof, expected) = stats.chi2_contingency(observed=month_sky)
chi, p, dof, expected

(1588.4404967678356,
 0.0,
 22,
 array([[ 1558.91386702,   567.47499036,  1073.61114262],
        [ 1433.22643649,   521.72231926,   987.05124424],
        [ 2287.21893927,   832.59221242,  1575.18884831],
        [ 3283.949493  ,  1195.42153438,  2261.62897262],
        [ 4294.80770365,  1563.39359844,  2957.79869791],
        [ 3419.86729578,  1244.8982601 ,  2355.23444412],
        [ 5208.72095819,  1896.07581154,  3587.20323027],
        [ 6049.56012522,  2202.15763446,  4166.28224032],
        [ 5626.70473879,  2048.23004333,  3875.06521788],
        [ 5112.26316267,  1860.96329651,  3520.77354082],
        [ 2989.7045006 ,  1088.31062995,  2058.98486945],
        [ 1686.0627793 ,   613.75966926,  1161.17755144]]))

In [147]:
(month_sky - expected)/(expected**.5)

SKY,No Cloud,Overcast,Some Cloud
INCIDENT_MONTH,,,
1,-3.568968,14.840440,-6.488772
2,-2.119142,7.586176,-2.961777
3,-0.422770,7.499921,-4.943198
4,0.733792,2.446241,-2.662701
5,-2.484295,2.013325,1.529839
6,-3.178325,-4.560206,7.145274
7,-3.570953,-9.118966,10.932728
8,-4.828559,-7.035535,10.933437
9,6.509617,-7.363000,-2.491007
